In [1]:
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models.feature_extraction import create_feature_extractor

import os

Loading model

In [2]:
from MyModel import MyResNet

In [3]:
model = MyResNet()
model

MyResNet(
  (pretrained): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(

Getting heatmap from output from transition layer & prediction layer weights

In [4]:
test_input = torch.rand(16, 3, 32, 32)

In [7]:
# getting output from transition layer
return_nodes = {
    "transition": "transition"
}
model2 = create_feature_extractor(model, return_nodes=return_nodes)

intermediate_outputs = model2(test_input)

Proxy(getattr_1)
Proxy(getattr_2)
Proxy(getattr_4)
Proxy(getattr_5)
Proxy(getattr_1)
Proxy(getattr_2)
Proxy(getattr_4)
Proxy(getattr_5)


In [24]:
###### multiplication #####
# original shapes: torch.Size([8, 2048]) and torch.Size([16, 2048, 32, 32])
result = intermediate_outputs['transition'].permute(0, 2, 3, 1) @ torch.transpose(model.prediction.weight, 0, 1)

In [32]:
result.shape

torch.Size([16, 32, 32, 8])

---

In [6]:
df = pd.read_csv('mimic_validation.csv')

In [7]:
df.head(3)

,dicom_id,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2a280266-c8bae121-54d75383-cac046f4-ca37aa16,10000898,50771383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
